# Reading the data

In [ ]:
# connecting the colab file from drive
from google.colab import drive
from tqdm import tqdm
drive.mount("/content/drive")
%cd '/content/drive/My Drive/Research/Philantrophy Research'

# reading file
import pandas as pd
df = pd.read_csv("Orphan_tweets_COVID-19.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Research/Philantrophy Research


# Data Pre-processsing

In [ ]:
!pip install emot
!pip install emoji
import re
import pickle
from emot import *
import emoji
from emot.emo_unicode import EMOJI_UNICODE
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
from tqdm import tqdm

# converting emojis to word
def convert_emojis_to_word(selected_review):
  for i in tqdm(range(len(selected_review))):
    #print(type(selected_review[i]))
    review=str(selected_review[i])
    selected_review[i]=emoji.demojize(review, delimiters=("", " "))
  return selected_review

# Converting emoticons to word
# adopted from https://stackoverflow.com/questions/70935501/unbalanced-parenthesis-regex
def convert_emoticons_to_word(selected_review):
  for i in tqdm(range(0,len(selected_review))):
    review=str(selected_review[i])
    for emot in EMOTICONS_EMO:
      review= review.replace(emot, EMOTICONS_EMO[emot].replace(" "," "))
    selected_review[i]=review
  return selected_review

# executing the functions that convert emojis and emoticons
df["text"]=convert_emojis_to_word(df["text"])
df["text"]=convert_emojis_to_word(df["text"])

# removing punctuation and special characters
df["text"] = df['text'].str.replace('[^A-Za-z ]','',regex=True) ## note: all Tweets have already been lowered case as part of the cleaning process

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/415 [00:00<?, ?it/s]<ipython-input-10-01fdb46f5a7b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_review[i]=emoji.demojize(review, delimiters=("", " "))
100%|██████████| 415/415 [00:00<00:00, 2623.50it/s]


# Sentiment Classification

In [ ]:
# installing flair
!pip install flair
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
from flair.data import Sentence

# Predicting the sentiments of the text using FLAIR 
def score_flair(text):
    sentence = Sentence(text) #takes the sentence 
    classifier.predict(sentence) #predicts the sentiment 
    score = sentence.labels[0].score # Getting the confidence of the sentiment 
    value = sentence.labels[0].value # Getting the sentiment 
    return score, value


# Get sentiment score for each review
df['sentiment_score_flair'] = df['text'].apply(lambda s: score_flair(s)[0])
# Predict sentiment label for each review
df['sentiment_NEG'] = df['text'].apply(lambda s: score_flair(s)[1])
df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-01-04 18:07:40,194 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang,time,sentiment_score_flair,sentiment_NEG
0,1250946636454518786,1117888059691995136,2020-04-17T00:38:14Z,sixty8eyes,just like germany paid reparations for holocau...,Twitter for iPhone,NaN,NaN,NaN,False,...,NaN,20,835,NaN,2019-04-15T20:30:57Z,False,en,2020-04-17,0.987752,POSITIVE
1,1251020605140410368,148455969,2020-04-17T05:32:09Z,sacsaraf,cont pmoindia amitshah nsitharaman nsitharaman...,Twitter for Android,NaN,NaN,NaN,False,...,NaN,33,121,NaN,2010-05-26T19:02:40Z,False,en,2020-04-17,0.999850,NEGATIVE
2,1251040259695050752,134349464,2020-04-17T06:50:15Z,DotunAdesua,may soocial media giveaways not replace giving...,Twitter for Android,NaN,NaN,NaN,False,...,NaN,1465,209,NaN,2010-04-18T04:59:13Z,False,en,2020-04-17,0.528027,NEGATIVE
3,1251044932191686656,956160317763018753,2020-04-17T07:08:49Z,SueGile56179408,currently reading houseoforphans by helen dunm...,Twitter for Android,NaN,NaN,NaN,False,...,NaN,10,153,NaN,2018-01-24T13:42:39Z,False,en,2020-04-17,0.995088,POSITIVE
4,1251098786765537281,780499166896291841,2020-04-17T10:42:49Z,PrabhasFanArmy,prabhas fans from podalakuru nellore dist dist...,Twitter for Android,NaN,NaN,NaN,False,...,NaN,8999,0,NaN,2016-09-26T20:07:57Z,False,en,2020-04-17,0.983888,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,1245823451614654465,228487825,2020-04-02T21:20:31Z,IsraelNitzan,israelis raise half a million dollars in hour...,Twitter for iPhone,NaN,NaN,NaN,False,...,NaN,7692,6951,NaN,2010-12-19T21:37:38Z,True,en,2020-04-02,0.909295,NEGATIVE
411,1245825084364857350,1316869394,2020-04-02T21:27:01Z,SahraRazi,we are instructing are orphans and widows to c...,Twitter for iPhone,NaN,NaN,NaN,True,...,NaN,8,2,NaN,2013-03-30T17:34:15Z,False,en,2020-04-02,0.991975,NEGATIVE
412,1246778971364167682,2464329458,2020-04-05T12:37:25Z,MediaEye360,hydroxichloroqueen billgates of hell gkillshot...,Twitter Web App,NaN,NaN,NaN,False,...,NaN,731,936,NaN,2014-04-26T08:50:07Z,False,fi,2020-04-05,0.999680,NEGATIVE
413,1246841220934098944,1174608015095451648,2020-04-05T16:44:46Z,fdrazi_org,our orphans and widows are taking nigellaseeds...,Twitter for iPhone,NaN,NaN,NaN,False,...,NaN,20,2,NaN,2019-09-19T08:56:01Z,False,en,2020-04-05,0.988847,POSITIVE


In [ ]:
# Counting the number of positive vs negative scores
df['sentiment_NEG'].value_counts()

POSITIVE    218
NEGATIVE    197
Name: sentiment_NEG, dtype: int64

In [ ]:
# Counting the mean confidence - it seems pretty high which is a good thing (ie the classifier is confident of the sentiments)
df["sentiment_score_flair"].mean()

0.9134345744029585

# Emotion Classification

In [ ]:
# installing NRCLex

!pip install NRCLex
# classifying tweets emotions
from nrclex import NRCLex
import nltk
nltk.download('punkt')

# Extracting the emotions 
def top_emotion_extractor(top_emotion):
  emotions=[]
  for i in top_emotion:
    emotions.append(str(i[0]))
  return(emotions)

# returning the list of emotions identified
def return_bin(emotion,list_of_emo):
  if emotion in list_of_emo:
    boo=1
  else:
    boo=0
  return(boo)

# Returning emotions of "fear", "anger", "sadness" and "disgust"
def emotion_classifier(text):
  emotions=top_emotion_extractor(NRCLex(text).top_emotions)
  fear=return_bin("fear",emotions)
  anger=return_bin("anger",emotions)
  sadness=return_bin("sadness",emotions)
  disgust=return_bin("disgust",emotions)
  return (fear,anger,sadness,disgust)

# Get emotion for each review
df['fear'] = df['text'].apply(lambda s: emotion_classifier(s)[0])
df['anger'] = df['text'].apply(lambda s: emotion_classifier(s)[1])
df['sadness'] = df['text'].apply(lambda s: emotion_classifier(s)[2])
df['disgust'] = df['text'].apply(lambda s: emotion_classifier(s)[3])
df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,account_created_at,verified,lang,time,sentiment_score_flair,sentiment_NEG,fear,anger,sadness,disgust
0,1250946636454518786,1117888059691995136,2020-04-17T00:38:14Z,sixty8eyes,just like germany paid reparations for holocau...,Twitter for iPhone,NaN,NaN,NaN,False,...,2019-04-15T20:30:57Z,False,en,2020-04-17,0.987752,POSITIVE,0,0,1,1
1,1251020605140410368,148455969,2020-04-17T05:32:09Z,sacsaraf,cont pmoindia amitshah nsitharaman nsitharaman...,Twitter for Android,NaN,NaN,NaN,False,...,2010-05-26T19:02:40Z,False,en,2020-04-17,0.999850,NEGATIVE,1,0,1,0
2,1251040259695050752,134349464,2020-04-17T06:50:15Z,DotunAdesua,may soocial media giveaways not replace giving...,Twitter for Android,NaN,NaN,NaN,False,...,2010-04-18T04:59:13Z,False,en,2020-04-17,0.528027,NEGATIVE,0,0,0,0
3,1251044932191686656,956160317763018753,2020-04-17T07:08:49Z,SueGile56179408,currently reading houseoforphans by helen dunm...,Twitter for Android,NaN,NaN,NaN,False,...,2018-01-24T13:42:39Z,False,en,2020-04-17,0.995088,POSITIVE,1,1,1,1
4,1251098786765537281,780499166896291841,2020-04-17T10:42:49Z,PrabhasFanArmy,prabhas fans from podalakuru nellore dist dist...,Twitter for Android,NaN,NaN,NaN,False,...,2016-09-26T20:07:57Z,False,en,2020-04-17,0.983888,POSITIVE,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,1245823451614654465,228487825,2020-04-02T21:20:31Z,IsraelNitzan,israelis raise half a million dollars in hour...,Twitter for iPhone,NaN,NaN,NaN,False,...,2010-12-19T21:37:38Z,True,en,2020-04-02,0.909295,NEGATIVE,1,1,1,0
411,1245825084364857350,1316869394,2020-04-02T21:27:01Z,SahraRazi,we are instructing are orphans and widows to c...,Twitter for iPhone,NaN,NaN,NaN,True,...,2013-03-30T17:34:15Z,False,en,2020-04-02,0.991975,NEGATIVE,1,0,0,0
412,1246778971364167682,2464329458,2020-04-05T12:37:25Z,MediaEye360,hydroxichloroqueen billgates of hell gkillshot...,Twitter Web App,NaN,NaN,NaN,False,...,2014-04-26T08:50:07Z,False,fi,2020-04-05,0.999680,NEGATIVE,1,1,1,1
413,1246841220934098944,1174608015095451648,2020-04-05T16:44:46Z,fdrazi_org,our orphans and widows are taking nigellaseeds...,Twitter for iPhone,NaN,NaN,NaN,False,...,2019-09-19T08:56:01Z,False,en,2020-04-05,0.988847,POSITIVE,0,0,0,0


In [ ]:
(df.sort_values(by=['created_at'])).head(50)

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,account_created_at,verified,lang,time,sentiment_score_flair,sentiment_NEG,fear,anger,sadness,disgust
215,1244078921869647872,330510566,2020-03-29T01:48:23Z,Zaheer_Bux,humans dont deserve the world we have bombed h...,Twitter for Android,NaN,NaN,NaN,False,...,2011-07-06T18:42:02Z,False,en,2020-03-29,0.999665,NEGATIVE,0,0,0,0
216,1244081073841209346,348092647,2020-03-29T01:56:56Z,pdhmin,doom to those who write wicked decrees and kee...,Twitter for Android,NaN,NaN,NaN,False,...,2011-08-03T21:30:38Z,False,en,2020-03-29,0.996239,NEGATIVE,0,0,0,0
217,1244128158804140032,355989081,2020-03-29T05:04:02Z,ANI,indira canteens functioning here will distrib...,Twitter Web App,NaN,NaN,NaN,False,...,2011-08-16T05:23:41Z,True,en,2020-03-29,0.939405,POSITIVE,0,0,0,0
218,1244142260448616448,785154787683790848,2020-03-29T06:00:04Z,IbadinSandra,at the end of this pandemic so many people aro...,Twitter for iPhone,NaN,NaN,NaN,False,...,2016-10-09T16:27:43Z,False,en,2020-03-29,0.567353,POSITIVE,1,0,1,0
219,1244162435294138371,823968755067797504,2020-03-29T07:20:14Z,Xenohadi,indira canteens functioning here will distrib...,IFTTT,NaN,NaN,NaN,False,...,2017-01-24T19:00:34Z,False,en,2020-03-29,0.916482,POSITIVE,0,0,0,0
220,1244211040201646081,836815544,2020-03-29T10:33:22Z,FrPhilTangorra,i am currently incapable of getting out to sen...,Twitter for Android,NaN,NaN,NaN,True,...,2012-09-21T02:25:45Z,False,en,2020-03-29,0.994971,NEGATIVE,0,0,0,0
221,1244232591055740928,1231667416427585537,2020-03-29T11:59:01Z,towotproject,towot leadership covid coronavirus preventivem...,Twitter Web App,1.244233e+18,1.231667e+18,towotproject,False,...,2020-02-23T19:49:57Z,False,und,2020-03-29,0.986971,POSITIVE,0,0,0,0
222,1244237503525277697,1594012968,2020-03-29T12:18:32Z,apehpeter,nathanielblow i agree can you kindly partner w...,Twitter for Android,1.244235e+18,2.109584e+08,nathanielblow,False,...,2013-07-14T18:55:58Z,False,en,2020-03-29,0.850529,POSITIVE,0,0,0,0
223,1244244432922136578,917396903880306688,2020-03-29T12:46:04Z,RevuriSrinivas9,i humbly req ktrtrs ministerktr garu pls kin...,Twitter for Android,NaN,NaN,NaN,False,...,2017-10-09T14:30:41Z,False,en,2020-03-29,0.776243,NEGATIVE,0,0,0,0
224,1244258488387338241,1594012968,2020-03-29T13:41:55Z,apehpeter,bukolasaraki good day sir would you like to pa...,Twitter for Android,1.244227e+18,1.570114e+08,bukolasaraki,False,...,2013-07-14T18:55:58Z,False,en,2020-03-29,0.817308,POSITIVE,0,0,0,0


# Organizing, Aggregating and saving the data

In [ ]:
# counting and aggregating the counts of Tweets
df_orphan=pd.DataFrame(df['time'].value_counts().sort_index()).rename(columns={"time": "Tweets"})

# This function aggregate the specified sentiments and emotions by date
def sentiment_count(sentiment,value,df):
  df_sentiment=df[df[sentiment]==value]
  s = df_sentiment['time'].value_counts().sort_index()
  return(pd.DataFrame(s).rename(columns={"time": sentiment}))
df_orphan2=sentiment_count("sentiment_NEG","NEGATIVE",df)
df_orphan=df_orphan.join(df_orphan2)
for i in ["fear","anger","sadness","disgust"]:
  data=sentiment_count(i,1,df)
  df_orphan=df_orphan.join(data)

df_orphan=df_orphan.fillna(0)
df_orphan=df_orphan.astype('Int64')
df_orphan.to_csv("sentimental_count_test_new.csv")
df_orphan

,Tweets,sentiment_NEG,fear,anger,sadness,disgust
2020-03-29,20,7,3,0,2,0
2020-03-30,14,7,4,2,2,1
2020-03-31,17,8,6,2,3,2
2020-04-01,14,8,2,2,5,2
2020-04-02,13,6,8,4,7,1
2020-04-03,15,9,4,3,3,2
2020-04-04,7,3,1,0,1,0
2020-04-05,3,2,1,1,1,1
2020-04-06,12,9,3,1,1,1
2020-04-07,11,8,4,2,5,2
